Sławomir Zieliński 2021

Bialystok University of Technology

## **Loop-Based Testing**

In [1]:
import os, shutil
import time, datetime, pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GroupKFold, LeaveOneGroupOut, GroupShuffleSplit, GridSearchCV, cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn import neighbors, linear_model, ensemble, svm
from sklearn import metrics
from xgboost import XGBClassifier, plot_importance, plot_tree, to_graphviz

# =========== Experiment type ===========
experiment_type = 'all'
classifier_type = 'xgboost'
test_type = 'hrtf-independent'


data_fname_blauert_bb = 'Blauert_Boosted_Bands_Features.csv'
data_fname_blauert_db = 'Blauert_Directional_Bands_Features.csv'
data_fname_hebrank_wright = 'Hebrank_and_Wright_Features.csv'
data_fname_langendijk_bronkhorst = 'Langendijk_and_Bronkhorst_Features.csv'

data_fname_lfcc = 'LFCC_30_Coeffs_Features.csv'
data_fname_binaural = 'Binaural_Features.csv'

data_dir = 'D:/FB Data/CSV'
results_dir = 'C:/Users/slawek/OneDrive/My Documents/Publications/2021/FB Paper/Results'
y_data_dir = 'C:/Users/slawek/OneDrive/My Documents/Publications/2021/FB Paper/y_data'
fig_dir = 'C:/Users/slawek/OneDrive/My Documents/Publications/2021/FB Paper/Figures'

n_scenes = 2 # 2 for 2-scenes scenario (front and back)
y_categories = [1, 2]
scene_labels = ['front', 'back']
labels_dict = {1:'front',2:'back'}

n_iterations = 7

## Read and Convert Raw Data

In [2]:
# Read Blauert's boosted bands features first
db_raw = pd.read_csv(os.path.join(data_dir, data_fname_blauert_bb))

# Convert the first variable to category type
db_raw['fileNames'] = db_raw['fileNames'].astype('category')

print('Raw data shape:', db_raw.shape)

Raw data shape: (22496, 41)


In [3]:
# Create auxiliary variables for db_raw set
filenames = db_raw['fileNames'] 
recordings = []
hrtf_ids = []
hrtf_sources = [] # Institution, organization, e.g. MIT
scene_ids = []
scenes = []
augmentation_numbers = []

for filename in filenames:
    # print(filename)
    filename_split = filename.split('_')
    recordings.append(filename_split[0])
    if len(filename_split[1]) == 5:
        hrtf_ids.append(filename_split[1][-1])
    elif len(filename_split[1]) == 6:
        hrtf_ids.append(filename_split[1][-2:])
    hrtf_sources.append(filename_split[2])
    scene_ids.append(filename_split[3][-1])
    scenes.append(filename_split[4])
    augmentation_numbers.append(filename_split[5][-5])

# Create dataframe
descriptive_variables = pd.DataFrame(
    {'file_name': filenames,
     'recording': recordings,
     'hrtf_id': hrtf_ids,
     'hrtf_source': hrtf_sources,
     'scene_id': scene_ids,
     'scene': scenes,
     'augmentation': augmentation_numbers
    }, dtype='category')

db_raw = descriptive_variables.join(db_raw)
db_raw.drop(['fileNames'], inplace=True, axis=1) # Drop reduntant column
db_raw

,file_name,recording,hrtf_id,hrtf_source,scene_id,scene,augmentation,bb_x_m_1,bb_x_m_2,bb_x_m_3,...,bb_mid_sd_1,bb_mid_sd_2,bb_mid_sd_3,bb_mid_sd_4,bb_mid_sd_5,bb_side_sd_1,bb_side_sd_2,bb_side_sd_3,bb_side_sd_4,bb_side_sd_5
0,AClassicEducationNightOwl_hrtf10_listen_scene1...,AClassicEducationNightOwl,10,listen,1,front,1,-13.366475,-14.920718,-17.737867,...,3.802044,2.269497,3.698789,8.505135,9.932851,2.200164,1.836064,2.989783,5.693024,10.150461
1,AClassicEducationNightOwl_hrtf10_listen_scene2...,AClassicEducationNightOwl,10,listen,2,back,1,-13.165625,-10.414157,-17.307292,...,3.798409,2.320676,3.461247,8.166631,10.029490,2.595530,1.649154,2.443347,5.334224,10.355624
2,AClassicEducationNightOwl_hrtf11_listen_scene1...,AClassicEducationNightOwl,11,listen,1,front,1,-12.762384,-14.009565,-16.179163,...,3.798318,2.425447,2.874754,5.606093,10.087953,2.349373,2.066034,3.479336,8.237912,9.867481
3,AClassicEducationNightOwl_hrtf11_listen_scene2...,AClassicEducationNightOwl,11,listen,2,back,1,-13.304450,-10.311370,-18.300599,...,3.805799,2.346021,2.972628,5.591150,9.834396,2.514024,2.078017,3.515702,8.195845,10.214499
4,AClassicEducationNightOwl_hrtf12_listen_scene1...,AClassicEducationNightOwl,12,listen,1,front,1,-13.017466,-12.899031,-13.463031,...,3.852681,2.257683,3.522456,8.963307,10.148667,3.760871,2.349555,2.855867,4.746690,9.648333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22491,YouKnowBetter_hrtf7_sadie_scene2_back_aug1.wav,YouKnowBetter,7,sadie,2,back,1,-10.551120,-18.346065,-27.321429,...,3.889259,4.557927,7.550142,9.794471,11.084363,4.029261,3.950695,7.014002,7.092333,11.818726
22492,YouKnowBetter_hrtf8_sadie_scene1_front_aug1.wav,YouKnowBetter,8,sadie,1,front,1,-9.751750,-17.510771,-26.927838,...,3.901691,4.580585,7.816229,8.925929,12.091808,3.508550,3.957277,6.589893,9.443778,11.187595
22493,YouKnowBetter_hrtf8_sadie_scene2_back_aug1.wav,YouKnowBetter,8,sadie,2,back,1,-9.196234,-15.621913,-26.735118,...,3.868315,4.845464,7.853752,9.887387,10.278599,3.547088,3.853594,5.815141,7.218115,12.414284
22494,YouKnowBetter_hrtf9_sadie_scene1_front_aug1.wav,YouKnowBetter,9,sadie,1,front,1,-9.753131,-18.969621,-25.159916,...,3.850202,4.237472,7.071604,7.851948,10.478471,3.737273,3.613938,6.905928,9.591553,11.980250


In [4]:
# Create additional auxiliary variables origin sources ('families') of HRTFs
hrtf_ids = db_raw['hrtf_id'].cat.categories.values
hrtf_sources = db_raw['hrtf_source'].cat.categories.values
hrtf_sources

array(['aachen', 'ari', 'cipic', 'clubfritz', 'hutubs', 'listen', 'mit',
       'riec', 'sadie', 'scut', 'th-koln', 'tu-berlin', 'viking'],
      dtype=object)

In [5]:
# # Remove Blauert's boosting bands
# db_raw.drop(db_raw.columns[np.arange(7,47)], axis=1, inplace=True)
# db_raw

# Read remaining datasets
db_raw_blauert_db = pd.read_csv(os.path.join(data_dir, data_fname_blauert_db))
db_raw_hebrank_wright = pd.read_csv(os.path.join(data_dir, data_fname_hebrank_wright))
db_raw_langendijk_bronkhorst = pd.read_csv(os.path.join(data_dir, data_fname_langendijk_bronkhorst))
db_raw_lfcc = pd.read_csv(os.path.join(data_dir, data_fname_lfcc))
db_raw_binaural = pd.read_csv(os.path.join(data_dir, data_fname_binaural))

db_raw_blauert_db.drop(['fileNames'], inplace=True, axis=1) # Drop reduntant column
db_raw_hebrank_wright.drop(['fileNames'], inplace=True, axis=1) # Drop reduntant column
db_raw_langendijk_bronkhorst.drop(['fileNames'], inplace=True, axis=1) # Drop reduntant column
db_raw_lfcc.drop(['fileNames'], inplace=True, axis=1) # Drop reduntant column
db_raw_binaural.drop(['fileNames'], inplace=True, axis=1) # Drop reduntant column

# Concatenate datasets
db_raw = db_raw.join([db_raw_blauert_db, db_raw_hebrank_wright, db_raw_langendijk_bronkhorst, db_raw_lfcc, db_raw_binaural])
# db_raw = db_raw.join([db_raw_blauert_db, db_raw_hebrank_wright, db_raw_langendijk_bronkhorst, db_raw_lfcc])
# db_raw = db_raw.join([db_raw_blauert_db, db_raw_hebrank_wright, db_raw_langendijk_bronkhorst, db_raw_binaural])
# db_raw = db_raw.join([db_raw_lfcc, db_raw_binaural])
db_raw

,file_name,recording,hrtf_id,hrtf_source,scene_id,scene,augmentation,bb_x_m_1,bb_x_m_2,bb_x_m_3,...,ic_sd_ch_33,ic_sd_ch_34,ic_sd_ch_35,ic_sd_ch_36,ic_sd_ch_37,ic_sd_ch_38,ic_sd_ch_39,ic_sd_ch_40,ic_sd_ch_41,ic_sd_ch_42
0,AClassicEducationNightOwl_hrtf10_listen_scene1...,AClassicEducationNightOwl,10,listen,1,front,1,-13.366475,-14.920718,-17.737867,...,0.023611,0.019060,0.018155,0.017408,0.012751,0.013184,0.024904,0.028165,0.022075,0.014887
1,AClassicEducationNightOwl_hrtf10_listen_scene2...,AClassicEducationNightOwl,10,listen,2,back,1,-13.165625,-10.414157,-17.307292,...,0.017362,0.013030,0.020101,0.016498,0.025959,0.026172,0.028664,0.023875,0.019687,0.018540
2,AClassicEducationNightOwl_hrtf11_listen_scene1...,AClassicEducationNightOwl,11,listen,1,front,1,-12.762384,-14.009565,-16.179163,...,0.052630,0.035431,0.031084,0.032807,0.041117,0.033306,0.031166,0.041929,0.034703,0.029015
3,AClassicEducationNightOwl_hrtf11_listen_scene2...,AClassicEducationNightOwl,11,listen,2,back,1,-13.304450,-10.311370,-18.300599,...,0.034318,0.018447,0.031849,0.044973,0.026626,0.044303,0.044530,0.042258,0.034486,0.022939
4,AClassicEducationNightOwl_hrtf12_listen_scene1...,AClassicEducationNightOwl,12,listen,1,front,1,-13.017466,-12.899031,-13.463031,...,0.054138,0.044948,0.029968,0.015117,0.022467,0.026151,0.034298,0.043854,0.040078,0.036155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22491,YouKnowBetter_hrtf7_sadie_scene2_back_aug1.wav,YouKnowBetter,7,sadie,2,back,1,-10.551120,-18.346065,-27.321429,...,0.040642,0.038293,0.041630,0.036094,0.041333,0.042339,0.045099,0.047074,0.049699,0.037355
22492,YouKnowBetter_hrtf8_sadie_scene1_front_aug1.wav,YouKnowBetter,8,sadie,1,front,1,-9.751750,-17.510771,-26.927838,...,0.048071,0.044221,0.054130,0.049519,0.036272,0.031890,0.041308,0.055338,0.058114,0.058880
22493,YouKnowBetter_hrtf8_sadie_scene2_back_aug1.wav,YouKnowBetter,8,sadie,2,back,1,-9.196234,-15.621913,-26.735118,...,0.033712,0.039130,0.049905,0.037873,0.044492,0.044542,0.041452,0.044125,0.050740,0.057483
22494,YouKnowBetter_hrtf9_sadie_scene1_front_aug1.wav,YouKnowBetter,9,sadie,1,front,1,-9.753131,-18.969621,-25.159916,...,0.047812,0.040295,0.040516,0.041753,0.034353,0.038616,0.037940,0.040865,0.047759,0.049177


### Summary statistics¶

In [6]:
n_all_features = db_raw.shape[1] - 7
print('Total number of features:',n_all_features)

db_raw.iloc[:,:7].describe(include = 'all').T

Total number of features: 676


,count,unique,top,freq
file_name,22496,22496,YouKnowBetter_hrtf9_sadie_scene2_back_aug1.wav,1
recording,22496,152,YouKnowBetter,148
hrtf_id,22496,74,9,304
hrtf_source,22496,13,viking,2128
scene_id,22496,2,2,11248
scene,22496,2,front,11248
augmentation,22496,1,1,22496


## Train-Test Function

In [7]:
def random_train_test_split(recordings_list, seed):
    train_recordings, test_recordings = train_test_split(recordings_list, 
                                                     test_size=0.26,
                                                     random_state = seed)

    ind_train = db_raw['recording'].isin(train_recordings)
    ind_test = db_raw['recording'].isin(test_recordings)

    db_train = db_raw[ind_train]
    db_test = db_raw[ind_test]
    
    print('Train data shape:', db_train.shape)
    print('Test data shape:', db_test.shape)
    
    return db_train, db_test

## Cross-Validation Setup

In [8]:
def my_kfold_cv(seed):
    n_splits = 10
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cv_groups = []
    
    return cv, cv_groups

## SGD Classifier Instantiation and Grid-Search Setup

In [9]:
# def sgd_setup():
    
#     classifier_sgd = linear_model.SGDClassifier(loss='hinge',
#                                                 alpha = 0.0001,
#                                                 random_state = seed)
    
#     k_best = SelectKBest(f_classif)
      
#     pipeline = Pipeline(steps=[('k_best', k_best),
#                                ('scaler', StandardScaler()), 
#                                ('sgd', classifier_sgd)]) 
#     k = ['all']
#     alpha_values = [0.00001, 0.0001, 0.001]
    
#     grid_search_parameters = {'k_best__k': k,
#                               'sgd__alpha': alpha_values}    
    
    
#     print(grid_search_parameters)
#     xgboost = False
    
#     return pipeline, grid_search_parameters


## SVM Classifier Instantiation and Grid-Search Setup

In [10]:
# def svm_setup():

#     classifier_svm = svm.SVC(kernel = 'rbf',
#                              random_state = seed)

#     k_best = SelectKBest(f_classif)
      
#     pipeline = Pipeline(steps=[('k_best', k_best),
#                                ('scaler', StandardScaler()), 
#                                ('svm', classifier_svm)]) 

# #     k = ['all', int((3/4)*n_features/2), int((1/2)*n_features)]
# #     C_values = [0.1, 1, 10, 100, 1000]
# #     gamma_values = [1/n_features] * np.array([0.01, 0.1, 1, 10, 100])

#     k = ['all']
#     C_values = [0.1, 1, 10]
#     gamma_values = [1/n_features] 
    
#     grid_search_parameters = {'k_best__k': k,
#                               'svm__C': C_values,
#                               'svm__gamma': gamma_values}
    
#     print(grid_search_parameters)
#     xgboost = False
    
#     return pipeline, grid_search_parameters   


## Logistic Regression Classifier Instantiation and Grid-Search Setup

In [11]:
# def logit_setup():
#     print('\n\n============ Logit ===============')
#     classifier_logit = linear_model.LogisticRegression(penalty ='l1', 
#                                                        multi_class ='ovr', 
#                                                        solver = 'liblinear',
#                                                        intercept_scaling = 10,
#                                                        max_iter = 150,
#                                                        random_state = seed)
#     k_best = SelectKBest(f_classif)

#     pipeline = Pipeline(steps=[('k_best', k_best),
#                                ('scaler', StandardScaler()),
#                                ('logit', classifier_logit)]) 

#     # k = [200, 300, 500, 700]
#     # C_values = [0.1, 1, 10, 100]
   
#     k = ['all']
#     C_values = [0.01, 0.1, 1]
   
#     grid_search_parameters = {'k_best__k': k,
#                               'logit__C': C_values} 
#     print(grid_search_parameters)
#     xgboost_flag = False
    
#     return pipeline, grid_search_parameters 

## XGBoost Classifier Instantiation and Grid-Search Setup

In [12]:
def xgboost_setup():

    classifier_xgboost = XGBClassifier(random_state = seed,
                                       # tree_method = 'hist',
                                       tree_method = 'gpu_hist',
                                       # objective='multi:softmax',
                                       # num_class = 3,
                                       importance_type = 'gain')

    k_best = SelectKBest(f_classif)

    pipeline = Pipeline(steps=[('k_best', k_best),
                               ('scaler', StandardScaler()), 
                               ('xgboost', classifier_xgboost)]) 

    # k = [200, 300, 500, 700]
    # n_estimators = [100, 200, 500, 1000]

    k = ['all']
    n_estimators = [200]
    

    grid_search_parameters = {'k_best__k': k,
                              'xgboost__n_estimators': n_estimators}
    print(grid_search_parameters)
    xgboost_flag = True
    
    return pipeline, grid_search_parameters   


## Training function

In [13]:
def train(X_data, y_data, pipeline, grid_search_parameters, cv, cv_groups, xgboost_flag):
    
    if xgboost_flag == True: 
        n_jobs = 1
    else:                       
        n_jobs = 16

    grid_search = GridSearchCV(pipeline, 
                               grid_search_parameters, 
                               cv = cv, 
                               n_jobs = n_jobs,
                               scoring = 'accuracy',
                               # iid = False,
                               return_train_score = True)
    start_time = time.time()

    grid_search.fit(X_data, y_data)
    # grid_search.fit(X_data, y_data, groups = cv_groups)

    pd_results = pd.DataFrame(grid_search.cv_results_)
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']

    zipped = zip(means, stds, params)
    print('\nGrid search results:\n')
    for mean, std, params in sorted(zipped, key = lambda x: x[0]):
        print("%0.3f (+/-%0.03f) for %r"% (mean, std, params))

    print('\nBest parameters:', grid_search.best_params_)
    print('\nTrain CV acc: %0.3f  Test acc: %0.4f'% (grid_search.best_score_, grid_search.score(X_test, y_test)))
    print('\nTime elapsed (hh:mm:ss): ', datetime.timedelta(seconds=(time.time() - start_time)))
    
    return grid_search

## Testing

In [14]:
# Accuracy report
def accuracy_report(y, y_pred):
    acc_test = metrics.accuracy_score(y, y_pred)
    print('\nTest metrics:')
    # print('Accuracy:', round(acc_test,3))
    print('Accuracy:', acc_test)
    print(metrics.classification_report(y, y_pred))
    return acc_test

# Confusion matrix
def confusion_mat(y, y_pred, labels):
    cm = metrics.confusion_matrix(y, y_pred, labels=labels)
    df_cm = pd.DataFrame(cm, columns=labels, index =labels)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (6,4))
    sns.set(font_scale=1.4) # for label size
    sns.heatmap(df_cm, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g', square=True)
    plt.ylim(len(labels), 0) # To prevent axis clipping effect
    plt.show()
    
# # Save pickle
# def save_pickle(data_name, fname, y_data_dir, experiment_type):
#     pickle_fname = os.path.join(y_data_dir, experiment_type, fname+'.pkl')
#     output_file = open(pickle_fname,'wb')
#     pickle.dump(data_name, output_file)
#     output_file.close()

## Main Loop

In [15]:
start_time = time.time()
acc_test_across_iterations = []

for ii in range(n_iterations):
    
    print('\n\nIteration:',ii)
    
    seed = ii

    db_train, db_test = random_train_test_split(db_raw.recording.unique(), seed)   
    
    
    # HRTF-Independent Testing
   
    acc_test = []
    for hrtf_source in hrtf_sources:
        print('\n',hrtf_source)
        
        ind_train = db_train['hrtf_source'] == hrtf_source
        ind_test = db_test['hrtf_source'] == hrtf_source
        
        # ========== Selecting Feature Group ============
        selected_columns = np.array(range(7, 7+int(n_all_features))) # All features

        X_train, y_train = db_train.iloc[:,selected_columns], db_train.loc[:,'scene']
        X_test, y_test = db_test.iloc[:,selected_columns], db_test.loc[:,'scene']

        column_names = db_train.columns.values.tolist()

        n_features = len(selected_columns)
        print('Number of selected features:',n_features)
        print('X_train.shape =',X_train[~ind_train].shape)
        print('X_test.shape =',X_test[ind_test].shape)

        cv, cv_groups = my_kfold_cv(seed)

#         # sgd
#         pipeline, grid_search_parameters  = sgd_setup()
#         grid_search = train(X_train[~ind_train], y_train[~ind_train], pipeline, grid_search_parameters, cv, cv_groups, xgboost_flag = False)
               
#         # svm
#         pipeline, grid_search_parameters  = svm_setup()
#         grid_search = train(X_train[~ind_train], y_train[~ind_train], pipeline, grid_search_parameters, cv, cv_groups, xgboost_flag = False)

#         # Logit
#         pipeline, grid_search_parameters  = logit_setup()
#         grid_search = train(X_train[~ind_train], y_train[~ind_train], pipeline, grid_search_parameters, cv, cv_groups, xgboost_flag = False)

        # XGBoost 
        pipeline, grid_search_parameters  = xgboost_setup()
        grid_search = train(X_train[~ind_train], y_train[~ind_train], pipeline, grid_search_parameters, cv, cv_groups, xgboost_flag = True)

    
        # Predicting - Test data
        y_test_pred = grid_search.predict(X_test[ind_test])
        accuracy_report(y_test[ind_test], y_test_pred)
        # confusion_mat(y_test[ind_test], y_test_pred, scene_labels)
        acc_test.append(grid_search.score(X_test[ind_test], y_test[ind_test])) 
        
    acc_test_across_iterations.append(acc_test)
    
    print('\nMean accuracy:',np.mean(acc_test))    
    

# Convert to dataframe
acc_test_across_iterations = np.array(acc_test_across_iterations).T
acc_test_across_iterations_df = pd.DataFrame(acc_test_across_iterations, index=hrtf_sources)


# Save to CSV file
csv_full_fname = results_dir+'/'+experiment_type+' '+classifier_type+' '+test_type+'.csv'
acc_test_across_iterations_df.to_csv(csv_full_fname, index=True)   
    
    
print('\n+++++++ Total computation time (hh:mm:ss): ', datetime.timedelta(seconds=(time.time() - start_time)))
# print('Mean test accuracy:', np.mean(acc_test))  



Iteration: 0
Train data shape: (16576, 683)
Test data shape: (5920, 683)

 aachen
Number of selected features: 676
X_train.shape = (16128, 676)
X_test.shape = (160, 676)
{'k_best__k': ['all'], 'xgboost__n_estimators': [200]}

Grid search results:

0.994 (+/-0.002) for {'k_best__k': 'all', 'xgboost__n_estimators': 200}

Best parameters: {'k_best__k': 'all', 'xgboost__n_estimators': 200}

Train CV acc: 0.994  Test acc: 0.9758

Time elapsed (hh:mm:ss):  0:00:30.014134

Test metrics:
Accuracy: 0.93125
              precision    recall  f1-score   support

        back       0.92      0.95      0.93        80
       front       0.95      0.91      0.93        80

    accuracy                           0.93       160
   macro avg       0.93      0.93      0.93       160
weighted avg       0.93      0.93      0.93       160


 ari
Number of selected features: 676
X_train.shape = (15008, 676)
X_test.shape = (560, 676)
{'k_best__k': ['all'], 'xgboost__n_estimators': [200]}

Grid search result

In [16]:
np.mean(acc_test)

0.9384065934065933

In [17]:
acc_test_across_iterations_df.values.mean()

0.9440600470957613

In [18]:
acc_test_across_iterations

array([[0.93125   , 0.95      , 0.94375   , 0.93125   , 0.9625    ,
        0.94375   , 0.93125   ],
       [0.95535714, 0.95535714, 0.96071429, 0.95892857, 0.96964286,
        0.95535714, 0.96071429],
       [0.94821429, 0.94285714, 0.93571429, 0.93928571, 0.95535714,
        0.94642857, 0.94107143],
       [0.93392857, 0.96964286, 0.975     , 0.975     , 0.97321429,
        0.96785714, 0.94285714],
       [0.94642857, 0.94642857, 0.96071429, 0.96428571, 0.95892857,
        0.96071429, 0.93571429],
       [0.95357143, 0.95357143, 0.96607143, 0.95892857, 0.96607143,
        0.96607143, 0.95535714],
       [0.95625   , 0.9625    , 0.9625    , 0.96875   , 0.975     ,
        0.95      , 0.95625   ],
       [0.91964286, 0.93392857, 0.94642857, 0.95714286, 0.9375    ,
        0.93392857, 0.91071429],
       [0.96964286, 0.975     , 0.98035714, 0.97857143, 0.98035714,
        0.9875    , 0.96964286],
       [0.925     , 0.8875    , 0.925     , 0.9125    , 0.9375    ,
        0.91875   , 0.9